In [1]:
import re
from datasets import load_dataset,load_from_disk,concatenate_datasets

/home/gz/Documents/Final_version_SLM/.llm_venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:


dataset = load_dataset("starhopp3r/TinyChat")
# %%
dataset


DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 1000000
    })
})

In [3]:
dataset = dataset.shuffle(seed=42)
dataset = dataset['train'].select(range(12000))

In [4]:
len(dataset)
pairs = []

Utility  Functions


In [5]:

def clean_text(text):
    text = text.lower()

    tokens = re.findall(r"\w+(?:'\w+)*|[^\w\s]", text)
    tokens = " ".join(tokens)

    return tokens



def preprocess_func(batch, tokenizer,clean_text):
    texts = []
    for text in batch["text"]:
        parts = re.split(r"\[/?INST\]", text)

        dialogue = [p.strip() for p in parts if p.strip()]

        for index in range(len(dialogue)-1):
            prompt = dialogue[index]
            response = dialogue[index+1]
            
          
            texts.append(f"{tokenizer.bos_token} <user> {clean_text(prompt)} {tokenizer.eos_token} <bot> {clean_text(response)} {tokenizer.eos_token}")

    tokens = tokenizer(texts, padding=False, truncation=False, return_attention_mask=False,add_special_tokens=False)

   
    input_ids = [list(ids) for ids in tokens["input_ids"]]

    return {
        "input_ids": input_ids,
        "raw_text": texts
    }



from transformers import AutoTokenizer
# load the saved tokenizer 
tokenizer_path = '/home/gz/Documents/Full Pipeline(LLM)/Saved_tokenizer/t5_Tokinzer'

tokenizer = AutoTokenizer.from_pretrained(tokenizer_path,use_fast=True)

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


In [6]:
dataset

Dataset({
    features: ['text'],
    num_rows: 12000
})

In [7]:
ds = dataset.map(
    lambda batch: preprocess_func(batch, tokenizer,clean_text),
    batched=True,
    batch_size=800,
    num_proc=6,
    remove_columns=dataset.column_names
    
)


In [8]:
len(ds)

92613

In [9]:
ds[10001]['raw_text']

'<start> <user> would you like to talk about what is making you feel sad today <end> <bot> i think it helps to share feelings and stem them before they grow . <end>'

In [10]:
len(ds)

92613

In [11]:
ds = ds.rename_column('raw_text','text_sample')

In [12]:
path = '/home/gz/Documents/Full Pipeline(LLM)/Saved_Data/processedDataset'
dataset_2 = load_from_disk(path)

In [13]:
dataset_2

Dataset({
    features: ['input_ids', 'text_sample'],
    num_rows: 104161
})

In [14]:
len(dataset_2)

104161

In [15]:
dataset_2

Dataset({
    features: ['input_ids', 'text_sample'],
    num_rows: 104161
})

In [26]:
dataset_2[98001]['text_sample']

'<start> <user> what do you need when you see a blue flag during the race ? <end> <bot> allow the leader to overtake . <end>'

In [16]:
ds

Dataset({
    features: ['input_ids', 'text_sample'],
    num_rows: 92613
})

In [17]:
combine_dataset =  concatenate_datasets([dataset_2,ds])

In [18]:
combine_dataset

Dataset({
    features: ['input_ids', 'text_sample'],
    num_rows: 196774
})

In [19]:
for sample in combine_dataset:
    print(sample)
    break

{'input_ids': [32101, 32103, 497, 3, 6, 3, 354, 603, 3, 6, 149, 81, 352, 21, 3, 9, 360, 36, 277, 227, 2634, 3, 58, 32100, 32104, 25, 214, 24, 19, 24873, 68, 19, 310, 59, 207, 21, 69, 4639, 3, 5, 32100], 'text_sample': '<start> <user> say , jim , how about going for a few beers after dinner ? <end> <bot> you know that is tempting but is really not good for our fitness . <end>'}


In [20]:

lengths = [len(row['input_ids']) for row in combine_dataset]

# Compute stats
avg_len = sum(lengths) / len(lengths)
min_len = min(lengths)
max_len = max(lengths)

print(f"Number of sequences: {len(dataset)}")
print(f"Average tokens per sequence: {avg_len:.1f}")
print(f"Minimum tokens: {min_len}")
print(f"Maximum tokens: {max_len}")


Number of sequences: 12000
Average tokens per sequence: 43.7
Minimum tokens: 11
Maximum tokens: 413


In [21]:
processed_path = '/home/gz/Documents/Full Pipeline(LLM)/Saved_Data/Combineed_Pairs_Dataset'

combine_dataset.save_to_disk(processed_path)

Saving the dataset (1/1 shards): 100%|██████████| 196774/196774 [00:00<00:00, 2081554.15 examples/s]
